In [1]:
import tensorflow as tf
from tensorflow.keras.models import load_model
import pathlib
import cv2
import numpy as np

In [2]:
filepath = "saved_model/best-model-3_doencas/model"
#model = load_model(filepath, compile = True)

In [3]:

converter = tf.lite.TFLiteConverter.from_saved_model(filepath)
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

In [4]:
tflite_models_dir = pathlib.Path("tflite_models/")
tflite_models_dir.mkdir(exist_ok=True, parents=True)

In [5]:
tflite_model_quant_file = tflite_models_dir/"3_doencas_model_quant.tflite"
tflite_model_quant_file.write_bytes(tflite_quant_model)

2803440

In [7]:
interpreter_quant = tf.lite.Interpreter(model_path="tflite_models/3_doencas_model_quant.tflite")
interpreter_quant.allocate_tensors()

In [5]:
#doente_00.JPG
#milho_normal_1.jpg
img_test = cv2.imread("doente_00.jpg")
img_test = cv2.resize(img_test, (128, 128))
img_test = img_test.astype('float32')/255

# Classe 0 = Normal
# Classe 1 = Doente

img_test = np.array(img_test)
img_test = img_test.reshape(1, 128, 128, 3)

In [8]:
input_index = interpreter_quant.get_input_details()[0]["index"]
output_index = interpreter_quant.get_output_details()[0]["index"]

interpreter_quant.set_tensor(input_index, img_test)
interpreter_quant.invoke()
predictions = interpreter_quant.get_tensor(output_index)




In [9]:
predictions

array([[5.786448e-10, 1.000000e+00]], dtype=float32)

In [10]:
voalor_certeza = np.max(predictions)
classe = np.argmax(predictions)
print(voalor_certeza)
print(classe)

1.0
1
